<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/09/hw_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!pip install tensorflow=2.17.1 pillow
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [13]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image
import tensorflow as tf
from tensorflow import keras

In [3]:
model = keras.models.load_model("model_2024_hairstyle.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_2024_hairstyle.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpyrlqwnja'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139014824649904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139014824654832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139014823276816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139014823278224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139014824649552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139014824652896: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
!ls -lh model_2024_hairstyle.tflite # Q1

-rw-r--r-- 1 root root 77M Dec  7 16:40 model_2024_hairstyle.tflite


In [8]:
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index # Q2

13

In [11]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [12]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, target_size=(200, 200))

In [18]:
img_array = np.array(img, dtype='float32') / 255.0
img_array[0, 0, 0] # Q3

0.23921569

In [19]:
interpreter.set_tensor(input_index, [img_array])
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds # Q4

array([[0.8937741]], dtype=float32)